# BERTの学習・推論・判定根拠可視化
IMDbのポジネガ判定をBERTでやってみる  
分類タスク用のアダプターモジュールを追加してファインチューニングする  
また，Self-Attentionの重みを可視化し，推論で重要となる単語をハイライト

## IMDbデータを読み込み，DataLoaderを作成
7章と異なる点があるのでここで再実装
- Bert用のWordPieceを用いてサブワードに対応したTokenizerを使用
- 訓練データに含まれている単語ではなく，BERTが持つ全単語を使用
    - BERTEmbeddingモジュールでは全単語を使用する
    - bert-base-uncased-vocab.txt

In [29]:
import os, re, time, tqdm, string, random
import torch.nn as nn
import torch.optim as optim
import torchtext
from utils.bert import BertTokenizer, load_vocab
from IPython.display import HTML

data_dir = "../../datasets/ptca_datasets/chapter8"
imdb_dir = os.path.join(data_dir, "aclImdb")
vocab_dir = os.path.join(data_dir, "vocab")
weights_dir = os.path.join(data_dir, "weights")
vocab_save_path=os.path.join(vocab_dir, "bert-base-uncased-vocab.txt")
weights_save_path = os.path.join(weights_dir, "pytorch_model.bin")
config_save_path = os.path.join(weights_dir, "bert_config.json")
model_save_path = os.path.join(data_dir, "bert_fine_tuning_IMDb.pth")

In [2]:
# IMDbの前処理(7章と同じ)
def preprocessing_text(text):
    text = re.sub('<br />', '', text)
    
    # カンマ，ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p ==","):
            # ピリオドとカンマの前後にはスペースを入れる
            text = text.replace(p, f" {p} ")
        else:
            text = text.replace(p, " ")
    
    return text

# 違うのはTokenizerがサブワード対応＆BERTのボキャブラリを使用していること
tokenizer_bert = BertTokenizer(vocab_save_path)

def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    return tokenizer(text)

データを読み込んだ時の処理をTEXT，LABELとして用意  
max_length=256で，BERTに入力するとき`<PAD>`を入れて512単語にする  
(SEPで2文に分割することはしない)

In [3]:
max_length = 256

TEXT = torchtext.data.Field(
    sequential=True,
    tokenize=tokenizer_with_preprocessing,
    use_vocab=True,
    lower=True,
    include_lengths=True,
    batch_first=True,
    fix_length=max_length,
    init_token="[CLS]",
    eos_token="[SEP]",
    pad_token="[PAD]",
    unk_token="[UNK]"
)

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

IMDbを整形したtsvファイルを読み込み，Datasetにする

In [4]:
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path=imdb_dir,
    train="IMDb_train.tsv",
    test="IMDb_test.tsv",
    format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)]
)

train_ds, val_ds = train_val_ds.split(split_ratio=0.8, random_state=random.seed(1234))

In [5]:
#  単語->ID, ID->単語
vocab_bert, ids_to_tokens_bert = load_vocab(vocab_save_path)

# TEXT.vocabを生成するため適当なデータでvocabを作ってからstoiを上書き
# もう少しいい方法があるのでは？
TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.stoi = vocab_bert

TEXTに単語->IDであるボキャブラリを用意できたので，DataLoaderを作成

In [6]:
batch_size = 32

train_dl = torchtext.data.Iterator(
    train_ds, batch_size=batch_size, train=True
)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=batch_size, train=False, sort=False
)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False
)

dataloaders_dict = {"train": train_dl, "val": val_dl}

動作確認

In [7]:
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[  101,  5791,  2012,  ...,  1012,  1999,   102],
        [  101,  1996,  5436,  ...,  1996,  2785,   102],
        [  101,  1045,  2572,  ...,     0,     0,     0],
        ...,
        [  101,  2045,  2024,  ..., 11790,  2027,   102],
        [  101,  5745,  2466,  ...,     0,     0,     0],
        [  101,  2073,  1996,  ...,  4237,  1012,   102]]), tensor([256, 256,  99, 256, 154, 137, 256, 256, 148, 256, 194, 256, 256, 170,
        246, 256, 159, 165, 256, 179, 156, 113, 228, 256, 256, 256, 256, 256,
        256, 256, 107, 256]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 0, 1, 1, 1])


ミニバッチの1文目を確認してみる

In [8]:
text_minibatch_1 = batch.Text[0][1].numpy()
text = tokenizer_bert.convert_ids_to_tokens(text_minibatch_1)
print(text)

['[CLS]', 'the', 'plot', 'of', 'this', 'film', 'might', 'not', 'be', 'extraordinary', ',', 'but', 'what', 'makes', 'the', 'film', 'really', 'special', ',', 'are', 'its', 'characters', 'and', 'the', 'actors', 'who', 'play', 'them', 'of', 'course', '.', 'i', 'won', 't', 'go', 'into', 'the', 'details', 'of', 'the', 'plot', 'of', 'the', 'movie', ',', 'but', 'i', 'would', 'certainly', 'like', 'to', 'say', 'this', 'this', 'film', 'is', 'not', 'just', 'for', 'everyone', 'the', 'film', 'is', 'really', 'witty', 'and', 'you', 'need', 'to', 'be', 'equally', 'clever', 'to', 'get', 'all', 'the', 'satire', '.', 'if', 'you', 're', 'not', 'alert', 'even', 'for', 'a', 'second', ',', 'you', 'll', 'probably', 'end', 'up', 'missing', 'one', 'of', 'the', 'subtle', 'points', '.', 'the', 'movie', 'is', 'full', 'of', 'such', 'seemingly', 'trivial', 'but', 'witty', 'stuff', 'like', 'the', 'announcements', 'going', 'on', 'in', 'the', 'background', 'at', 'tu', '##ra', '##qi', '##stan', ',', 'the', 'advertisement

`##word`は前に繋がる単語に付随するサブワードを意味する

## 感情分析用のBERTモデルを構築
- 学習済みパラメータをロード
- ポジネガ分類用モジュールを取り付ける
- 感情分析を行う

In [24]:
from utils.bert import get_config, BertModel, set_learned_params

config = get_config(config_save_path)
net_bert = BertModel(config)
net_bert = set_learned_params(net_bert, weights_save_path)

BERTの基礎部分にポジネガ分類のための全結合層1つによるアダプタを取り付け  
BERTの先頭単語の特徴量はNSPにより入力文章全体の特徴を反映している

In [26]:
class BertForIMDb(nn.Module):
    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()
        
        self.bert = net_bert
        self.cls = nn.Linear(in_features=768, out_fetures=2)
        
        # ポジネガ分類モジュールだけ重みの正規分布初期化を行う
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None,
                output_all_encoded_layers=False, attention_show_flg=False):
        
        if attention_show_flg:
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask,
                output_all_encoded_layers, attention_show_flg
            )
        else:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask,
                output_all_encoded_layers, attention_show_flg
            )
        
        # 入力文章の最初の単語の部分を使用してポジネガ分類を行う
        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768)
        out = self.cls(vec_0)
        
        if attention_show_flg:
            return out, attention_probs
        else:
            return out

## Bertのファインチューニングに向けた設定
12段全てをファインチューニングすると時間がかかるので，最終段のみ訓練

In [ ]:
for name, param in net.named_parameters():
    param.requires_grad = False

for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

for name, param in net.cls.named_parameters():
    param.requires_grad = True

## 学習・検証を実施
attention_maskの`<PAD>`されている部分には意味がないとだいたい学習できているため，attention_maskをNoneにして全てにSelf-Attentionをかけてしまう

In [31]:
def train_model(device, net, dataloaders_dict, criterion, optimizer, num_epochs):
    net.to(device)
    torch.backends.cudnn.benchmark = True
    batch_size = dataloaders_dict["train"].batch_size
    
    for epoch in range(1, num_epochs+1):
        for phase in ['train', 'val']:
            if phase == "train":
                net.train()
            else:
                net.eval()
            
            epoch_loss = 0.0
            epoch_corrects = 0
            iteration = 1
            
            t_epoch_start = time.time()
            t_iter_start = time.time()
            
            for batch in (dataloaders_dict[phase]):
                inputs = batch.Text[0].to(device)
                labels = batch.Label.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        if (iteration % 10 == 0):
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            acc = (torch.sum(preds == labels.data))
                            acc = acc.double() / batch_size
                            print(f"Iteration {iteration} || Loss: {loss.item():.4f} || Acc: {acc:.4f} || {duration:.4f} sec")
                            t_iter_start = time.time()
                
                iteration += 1
                epoch_loss += loss.item() * batch_size
                epoch_corrects += torch.sum(preds == labels.data)
        
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            
            print(f"Epoch {epoch}/{num_epochs} | {phase:^5} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")
            t_epoch_start(time.time())

    return net

In [ ]:
num_epochs = 2

device = torch.device('cuda:0')
net = BertForIMDb(net_bert)
print('ネットワーク設定完了')

# ハイパーパラメータはBERTの論文で推奨されている値を使用
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 5e-5}
], betas = (0.9, 0.999))

criterion = nn.CrossEntropyLoss()

net_trained = train_model(
    device, 
    net, 
    dataloaders_dict, 
    criterion, 
    optimizer,
    num_epochs=num_epochs
)

パラメータを保存

In [ ]:
torch.save(net_trained.state_dict(), model_save_path)

ロード

In [ ]:
config = get_config(config_save_path)
net_bert = BertModel(config)
net = BertForIMDb(net_bert)
net.load_state_dict(torch.load(model_save_path))

テストデータで正解率を確認してみる

In [ ]:
device = torch.device("cuda:0")

net_trained.eval()
net_trained.to(device)

epoch_corrects = 0
for batch in tqdm(test_dl):
    inputs = batch.Text[0].to(device)
    labels = batch.Label.to(device)
    with torch.set_grad_enabled(False):
        outputs = net_trained(inputs)
    loss = criterion(outputs, labels)
    _, preds = torch.max(outputs, 1)
    epoch_corrects += torch.sum(preds == labels.data)

epoch_acc = epoch_corrects.double() / len(test_dl.dataset)
print(f"テストデータ{len(test_dl.dataset)}個での正解率： {epoch_acc}")

正解率が90%を超えた  
7章では80%だったので，大きく正答率が向上したと言える

## Attentionの可視化
Self-Attentionの重みを可視化し，推論  
テストデータの最初の64文章を推論してみる

In [ ]:
batch_size = 64
test_dl = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False)

batch = next(iter(test_dl))
inputs = batch.Text[0].to(device)
labels = batch.Label.to(device)

outputs, attention_probs = net_trained(inputs, attention_show_flg=True)
_, preds = torch.max(outputs, 1)

文章をAttentionの重みに応じて色つけするHTMLを作成  
7章とほぼ同じだが，次の点が異なる  
- 前回は初段と終段の2つだったが，今回は終段のAttentionのみ扱う
- multi-headedな出力それぞれについて可視化の結果を確認する

In [7]:
def highlight(word, attn):
    ''' Attentionの値が大きいと文字の背景が濃い赤になるHTMLを作成 '''
    # 16進数2文字で3つの値を出力
    color = '#%02X%02X%02X' % (255, int(255*(1-attn)), int(255*(1-attn)))
    html = f' <span style="background-color: {color}">{word}</span>'
    return html

def highlight_sentence(sentence, attns):
    ''' 各単語をAttentionの値に応じてハイライト, SEPが出てきたらそこで終わる '''
    html = ""
    for word, attn in zip(sentence, attns):
        word = [word.numpy().tolist()]
        word = tokenizer_bert.convert_ids_to_tokens(word)[0]
        if word == "[SEP]":
            break
        html += highlight(word, attn)
    return html

def mk_html(index, batch, preds, normalized_weights, TEXT):
    # indexの結果を抽出
    sentence = batch.Text[0][index]
    label = batch.Label[index]
    pred = preds[index]
    
    # ラベルと予測結果を文字に置き換え
    label_str = "Negative" if label == 0 else "Positive"
    pred_str = "Negative" if pred == 0 else "Positive"
    
    # HTMLの作成
    html = f"正解ラベル:{label_str}<br>推論ラベル:{pred_str}<br><br>"
    
    # 12個のMulti-Head Self-Attentionそれぞれの重みを可視化
    for i in range(12):
        
        # indexのAttentionを抽出し，規格化
        # 0単語目のi番目のMulti-Head Attentionを取り出す
        attens = normalized_weights[index, i, 0, :]
        attens /= attens.max()
        
        html += f"[BERTのAttention {i+1:^2} を可視化]<br>"
        html += highlight_sentence(sentence, attens)
        html += "<br><br>"
    
    # 全Attentionの重みの平均を可視化
    all_attens = attens * 0 # zeros like
    for i in range(12):
        all_attens += normalized_weights[index, i, 0, :]
    all_attens /= all_attens.max()
    
    html += '[BERTのAttention全体の平均を可視化]<br>'
    html += highlight_sentence(sentence, all_attens)
    html += "<br><br>"
    
    return html

うまく判定できている場合

In [ ]:
index = 3 # 可視化対象文章のID
html = mk_html(index, batch, preds, attention_probs, TEXT)
HTML(html)

7章でうまく判定できていなかった文章の場合

In [ ]:
index = 0 # 可視化対象文章のID
html = mk_html(index, batch, preds, attention_probs, TEXT)
HTML(html)